<a href="https://colab.research.google.com/github/arbi11/CompEM1/blob/master/DQN_Worm_DyNASTack_21_Sep_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pwd

'C:\\Users\\chetanm'

In [0]:
cd Desktop\Deep Learning\RL_Dynamic

C:\Users\chetanm\Desktop\Deep Learning\RL_Dynamic


In [0]:

M_NO                = 1
MACHINE             = 'Acer'
PATH 				        = 'C:\\Users\\chetanm\\Desktop\\Deep Learning\\RL_Dynamic'
#PATH                = 'C:\\Users\\Admin\\Desktop\\Arbi4'

MODEL_NAME          = 'Colab_Data_Eval_QL1N3_BAG_Dy'
TRAIN_NAME 			    = 'Colab_Data_Train_QL1N3_BAG_Dy'

max_steps           = 75
env_dim             = [21, 35]
action_dim          = 8
state_size          = (env_dim[0] + 1) * env_dim[1]
max_iron            = 190

lr                  = 3e-4
batch_size          = 2048
miniBatchSize       = 1024
penalty             = -2

gamma               = 0.95                             # Discounting rate
playTime            = 10

warmUpSteps         = 1000 #200
experienceLength    = 4000
hidden_size         = [512, 256]
variance            = 0.4
num_episodes        = 1500

max_eps             = 1.0
min_eps             = 0.01
decay_rate          = 0.001
 
tau                 = 0.001


In [0]:
import numpy as np
import femm
import imageio
import warnings
from  skimage import img_as_float, img_as_uint
import matplotlib.pyplot as plt
import re
import os
import constant_1N3 as con
from shutil import copyfile
import pickle

FEMM_PATH = con.PATH + '\\'
#FEMM_PATH = 'C:\\Users\\Arbi\\Desktop\\Arbi\\'
MODEL_PATH = con.MODEL_NAME + '\\'
TRAIN_PATH = con.TRAIN_NAME + '\\'

class WormFemmEnv3:

    def __init__(self, env_dim = [23, 36], startR=3 , startC= 4, max_steps= con.max_steps):
        
        self.env_dim = env_dim
        self.state_hist = []
        self.step_count = 0
        self.posR = startR
        self.posC = startC
        self.done = 0.0
        self.repeat = 0
        self.issue = 0
        self.w = 1
        self.max_steps = max_steps
        self.past_rewards = []
        self.past_states = []
        self.past_BnStates = []        
        self.past_actions = []   
        self.net_force = 0.0
        self.R = 0.0
        self.count = 0
        
        self.coil_start = 2
        self.coil_width = 3
        self.coil_length = 9
        
        self.coil_current_choice = np.array([1, 1.5])
        self.coil_turns_choice = np.array([500])
        
        self.arm_start = 27
        self.arm_width = 6
        self.arm_length = 15
        
        self.cont_start_col = self.coil_start + self.coil_width
        self.window_width = self.arm_start - (self.coil_start + self.coil_width)
        self.window_length = np.max([self.arm_length, 15])
        self.frame_idx = 0
        self.worm_step_size = 1
        self.exp_count = 0

        
    def create_geo(self):
        
        femm.openfemm(1);
        femm.opendocument(FEMM_PATH + 'actuator.fem');

        femm.mi_getmaterial('Air')
#        femm.mi_addmaterial('LinearIron', 2100, 2100, 0, 0, 0, 0, 0, 1, 0, 0, 0)
        femm.mi_getmaterial('Cold rolled low carbon strip steel')            
        femm.mi_getmaterial('Copper')
        femm.mi_addcircprop('icoil', self.coil_current, 1);
        
        # femm.mi_setblockprop('Air', 0, 10, '<None>', 0, 0, 0);
        femm.mi_clearselected();

        for i in range(17):
            for j in range(35):
                
                femm.mi_selectlabel(i + 0.5, j + 0.5);
                
        femm.mi_setblockprop('Air', 0, 0.5, '<None>', 0, 0, 0);
        femm.mi_clearselected();

        coil2_start = np.copy(self.coil2_start)
        femm.mi_clearselected();
        
        femm.mi_selectrectangle(0 + 0.1, self.coil_start + 0.1, self.coil_length - 0.1, self.coil_start + self.coil_width - 0.1, 4)
        femm.mi_deleteselectedlabels()
        femm.mi_deleteselectednodes()
        femm.mi_deleteselectedsegments()
        femm.mi_drawrectangle(0, self.coil_start, self.coil_length, self.coil_start + self.coil_width)
        femm.mi_addblocklabel((0 + self.coil_length)/ 2, (2*self.coil_start + self.coil_width)/ 2)
        
        femm.mi_clearselected();

        femm.mi_selectrectangle(0 + 0.1, coil2_start + 0.1, self.coil_length - 0.1, coil2_start + self.coil_width - 0.1, 4)
        femm.mi_deleteselectedlabels()
        femm.mi_deleteselectednodes()
        femm.mi_deleteselectedsegments()        
        femm.mi_drawrectangle(0, coil2_start, self.coil_length, coil2_start + self.coil_width)
        femm.mi_addblocklabel((0 + self.coil_length)/ 2, (2*coil2_start + self.coil_width)/ 2)
        femm.mi_clearselected();

        femm.mi_selectlabel((0 + self.coil_length)/ 2, (2*coil2_start + self.coil_width)/ 2);
        femm.mi_setblockprop('Copper', 0, 0, 'icoil', 0, 1, self.coil_turns); 
        femm.mi_clearselected();

        femm.mi_selectlabel((0 + self.coil_length)/ 2, (2*self.coil_start + self.coil_width)/ 2)
        femm.mi_setblockprop('Copper', 0, 0, 'icoil', 0, 1, -self.coil_turns); 
        
        #        femm.mi_setblockprop('Copper', 0, 0.25, 'icoil', 0, 1, 500);        
        femm.mi_clearselected();

        femm.mi_selectrectangle(0 + 0.1, self.arm_start + 0.1, self.arm_length - 0.1, self.arm_start + self.arm_width - 0.1, 4)
        femm.mi_deleteselectedlabels()
        femm.mi_deleteselectednodes()
        femm.mi_deleteselectedsegments()        
        femm.mi_drawrectangle(0, self.arm_start, self.arm_length, self.arm_start + self.arm_width)
        femm.mi_addblocklabel((0 + self.arm_length)/ 2, (2*self.arm_start + self.arm_width)/ 2)
        femm.mi_selectlabel((0 + self.arm_length)/ 2, (2*self.arm_start + self.arm_width)/ 2);
        femm.mi_setblockprop('Cold rolled low carbon strip steel', 0, 0.5, '<None>', 0, 5, 0);
        femm.mi_clearselected();
                
        femm.mi_drawrectangle(0, 100, 100, -100)
        femm.mi_addblocklabel(50, 0);
        femm.mi_selectlabel(50, 0);
        femm.mi_setblockprop('Air', 50, 0, '<None>', 0, 0, 0);
        femm.mi_clearselected();

        femm.mi_addboundprop('neumann', 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0)
        femm.mi_selectrectangle(-0.5, 100.5, 0.2, -100.5, 4)
        femm.mi_setsegmentprop('neumann', 0, 0, 0, 5)
        femm.mi_clearselected();       
        
        femm.mi_selectsegment(95, -95)
        femm.mi_selectsegment(55, 95)
        femm.mi_selectsegment(55, -95)
        femm.mi_addboundprop('dirichlet', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
        femm.mi_setsegmentprop('dirichlet', 0, 0, 0, 0)
        femm.mi_clearselected();       

        femm.mi_zoomnatural();
        femm.mi_saveas(FEMM_PATH + 'actuator2.fem');  
        femm.closefemm()
        
        #####################################################################################
        #                                   VARIABLES                                       #       
        #####################################################################################
        
        # step_count           :       step# in an episode.
        # done          :       episode over or not.
        # snap          :       store material dist as png (mat2gray).
        # act_py        :       action (material dist) sent to matlab.
        # state_abs     :       action with discrete values(0/1) & flattened.
        # changes       :       changes made between (n-1) & 'n'th actions.
        # act_mat       :       action as matlab double.
        # count         :       No. of iron (1's) in the action.
        # frame_idx     :       episode #.
        
        #####################################################################################
        # mat_res       :       output from matlab:                                         #
        #                               nargout= 2 (means 2 variables from matlab)          #
        # field_xy      :                   var1 = Bx, By (concatenated) dim: (2, 21, 15)   #
        # Fx            :                   var2 = Force magnitude dim: 1(scalar)           #
        #####################################################################################
        
        # reward        :       Reward for the agent (Fx(n) - Fx(n-1)).
        # state3        :       environment state for the agent. dim: (3, 21, 15).
        
        #####################################################################################
        
    def move(self, mat_state):
        if (self.worm_step_size == 3):
            mat_state[self.posR -1 : self.posR + 2, self.posC -1 : self.posC + 2] = 1    
            mat_state[self.posR , self.posC] = 5
        elif (self.worm_step_size == 1 and self.past_actions[-1]<4):
            mat_state[self.posR -1 : self.posR + 2, self.posC -1 : self.posC + 2] = 1    
            mat_state[self.posR , self.posC] = 5
        else:
            mat_state[self.posR , self.posC] = 5

        return(mat_state)

    def reward(self, mat_state):
        
        #print('entered REWARD process with a shape of:', mat_state.shape)

        femm.openfemm(1);
        femm.opendocument(FEMM_PATH + 'actuator2.fem');
        #femm.mi_addmaterial('LinearIron', 2100, 2100, 0, 0, 0, 0, 0, 1, 0, 0, 0)

        indices = np.nonzero(mat_state[self.buffer:-1, :] - self.state_hist[-1])
        
        len_ind = np.size(indices[0])
        for i in range(len_ind):
            #print(indices[0][i], ',', indices[1][i])
            femm.mi_selectlabel(indices[0][i] + 0.4, indices[1][i] + 0.4);
            femm.mi_deleteselectedlabels()
            femm.mi_addblocklabel(indices[0][i] + 0.4, indices[1][i] + 0.4)
            femm.mi_clearselected();

        for i in range(len_ind):
            femm.mi_selectlabel(indices[0][i] + 0.4, indices[1][i] + 0.4);
            femm.mi_setblockprop('Cold rolled low carbon strip steel', 0, 0, '<None>', 0, 3, 0);
            
        femm.mi_clearselected();
        femm.mi_zoomnatural()
        femm.mi_saveas('actuator2.fem')
        femm.mi_analyse(0)
        femm.mi_loadsolution()
                
        Bx, By = femm.mo_getb(0.2, 27.12)
        B = np.sqrt(Bx ** 2 + By ** 2)
        femm.mo_groupselectblock(5)
        self.net_force = femm.mo_blockintegral(19) *(-1)
                  
        Bxy_state = np.zeros([self.env_dim[0]-self.buffer, self.env_dim[1], 2])
        for r in range(Bxy_state.shape[0]):
            for c in range(self.env_dim[1]):
                Bxy_state[r, c, :] = femm.mo_getb(r+0.5, c+0.5)
            
        self.B_state = np.sum(np.square(Bxy_state), axis=-1)*1000        
                   
        femm.closefemm()
        
        reward = B*1000
                
        return(reward)
        
    def clean_up(self, mat_state):
        
        mat_state[self.buffer: self.buffer + self.coil_length, 
                  self.coil_start: self.coil_start + self.coil_width] = self.coil_turns * self.coil_current
        coil2_start = np.copy(self.coil2_start)
        mat_state[:, 0:self.coil_start] = 3
        mat_state[self.buffer: self.buffer + self.coil_length, 
                  coil2_start : coil2_start + self.coil_width] = self.coil_turns * self.coil_current
        
        mat_state[0:self.buffer, :] = 3      
        mat_state[:, self.arm_start- 1] = 3
        mat_state[self.window_length + self.buffer:, :] = 3
        mat_state[self.buffer: self.buffer + self.arm_length, 
                  self.arm_start : self.arm_start + self.arm_width] = 4
        
        mat_state[mat_state == 5] = 1
        mat_state[self.posR, self.posC] = 5

        return mat_state
    
    ##### ACTIONS #######
    # 0     : RIGHT     #
    # 1     : LEFT      #
    # 2     : UP        #
    # 3     : DOWN      #
    ##-----------------##
    # 4     : RIGHT     #
    # 5     : LEFT      #
    # 6     : UP        #
    # 7     : DOWN      #
    #####################
    
    def step(self, action):
        
        state = self.past_states[-1]
        self.step_count +=1
        self.R = 0.0
        self.count = 0

        if action == 0:
            self.worm_step_size = 1
            self.posC += self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 1:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.issue = 2
                self.posC -= self.worm_step_size
                        
        elif action == 1:
            self.worm_step_size = 1                               
            self.posC -= self.worm_step_size
            
            if state[self.posR, self.posC] == 0:
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 1:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.posC += self.worm_step_size
                self.issue = 2
        
        elif action == 2:
            self.worm_step_size = 1                               
            self.posR -= self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 1:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.posR += self.worm_step_size
                self.issue = 2
        
        elif action == 3:
            self.worm_step_size = 1    
            self.posR += self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 1:
                state2 = self.move(state)
                self.issue = 1
            else:
                self.posR -= self.worm_step_size
                self.issue = 2
        
        elif action == 4:
            if self.past_actions[-1]<4:
                self.worm_step_size = 1
            else:
                self.worm_step_size = 3  
                
            self.posC += self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 1:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.issue = 2
                self.posC -= self.worm_step_size
                        
        elif action == 5:
            if self.past_actions[-1]<4:
                self.worm_step_size = 1
            else:
                self.worm_step_size = 3                    
            
            self.posC -= self.worm_step_size
            
            if state[self.posR, self.posC] == 0:
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 1:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.posC += self.worm_step_size
                self.issue = 2
        
        elif action == 6:
            if self.past_actions[-1]<4:
                self.worm_step_size = 1
            else:
                self.worm_step_size = 3
                
            self.posR -= self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 1:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.posR += self.worm_step_size
                self.issue = 2
        
        elif action == 7:
            if self.past_actions[-1]<4:
                self.worm_step_size = 1
            else:
                self.worm_step_size = 3
            self.posR += self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 1:
                state2 = self.move(state)
                self.issue = 1
            else:
                self.posR -= self.worm_step_size
                self.issue = 2
                
        if self.issue == 0:    
            #print('Going for reward with a shape of:', state2.shape)
            new_state = self.clean_up(state2)
            try:
                F = self.reward(new_state)
            except:
                try:
                    F = self.reward(new_state)
                except:
                    F = self.reward(new_state)
                    
            self.R = F - self.past_rewards[-1]
#            R = F
            self.past_rewards.append(F)
           
        elif self.issue == 1:
                                 
#            self.R = con.penalty + 2
#            new_state = self.clean_up(state2)
#            self.past_rewards.append(F)
            new_state = self.clean_up(state2)
            try:
                F = self.reward(new_state)
            except:
                try:
                    F = self.reward(new_state)
                except:
                    F = self.reward(new_state)
            
            self.R = F - self.past_rewards[-1]
#            R = F
            self.past_rewards.append(F)
           
        elif self.issue == 2:
            
            new_state = state
#            self.R = constants_DQL.penalty
#            F = self.past_rewards[-1]
#            self.past_rewards.append(F)
            
            #print('NOT Going for REWARD with a shape of:', new_state.shape)
            #print('NOT Going for REWARD coz of state[',self.posR,',',self.posC, '] = ', state[self.posR, self.posC])
            
            if state[self.posR, self.posC] == 2:
                self.R = con.penalty
                # coil
            elif state[self.posR, self.posC] == 3:
                self.R = con.penalty - 1
                # outside air
            else:
                self.R = con.penalty
                # Repeat
                
        unique, counts = np.unique(new_state, return_counts=True)
        index_1 = np.where (unique == 1)
        index_5 = np.where (unique == 5)
        try:
            self.count = np.asscalar(counts[index_1[0]] + 1)
            
        except:
            self.count = 0        
        
        if self.count > con.max_iron or self.step_count == self.max_steps:
            self.done = 1.0
            #print("Steps:", self.step_count, "Iron Count:", count, "Reward", self.past_rewards[-1])
#            R = self.reward(new_state)
#            print("Steps:", self.step_count, "Iron Count:", count, "Reward", R)    
#        elif count > 80:
#            R += 1
#        elif count > 80 and self.issue == 2:
#            R = 0
#        print("Steps:", self.step_count, "Iron Count:", count, "Reward", R, "Action- ", action)
        
        if new_state[self.posR, self.posC] != 5:
            raise('Problem in code. Worm outside the geometry')
            
        elif np.asscalar(counts[index_5[0]]) > 1:
            raise('Problem in code. More than one worm heads <<MEDUSA>>')
        
        self.past_states.append(np.copy(new_state))
        self.past_actions.append(np.copy(action))
#        , "Changes:", changes-22
        
        new_state2 = new_state[self.buffer:, :]
        new_state2[:, self.arm_start-1] = self.B_state[:, self.arm_start-1]
        self.past_BnStates.append(np.copy(new_state2))                
        
        new_state3 = np.expand_dims(new_state2, 0)
		
#        fd = open(eps_dir + str(frame_idx) + '.csv','a')
#        writer=csv.writer(fd,delimiter=',',quoting=csv.QUOTE_MINIMAL)
#        writer.writerow([self.step_count, R, count])
#        fd.close()
#        new_state = np.expand_dims(new_state, axis= -1)    
#        state_index = np.argmax(new_state[1:16, 5:26])
        new_state4 = np.copy(new_state3)
        return(new_state4, self.R, self.done, {})
        
          
    def reset(self, geo_dynamic= False):
        
    #    state_dim = [21, 31]  # size of geometry

        self.worm_step_size = 3            
        self.past_rewards = []
        self.past_states = []
        self.past_actions = []
        self.state_hist = []        
        self.step_count = 0
        self.posC = 1+self.w
        self.posR = 1+self.w
        self.done = 0.0
        self.max_steps = con.max_steps
        self.past_rewards = []
        self.past_states = []
        self.net_force = 0.0
        self.R = 0.0
        self.count = 0         
        self.exp_count = 0      

        if geo_dynamic == True:
            self.coil_start = np.random.randint(1, 6)        
            self.arm_start = np.random.randint(25, 31)
            self.coil_width = np.random.randint(3, 8- self.coil_start + 1)
            self.coil2_start = np.random.randint(self.coil_start + self.coil_width + 4, self.coil_start + self.coil_width + 8)  
            self.arm_width = np. random.randint(3, 35 - self.arm_start)
            self.arm_length = np.random.randint(13, 17)
            self.window_length = np.random.randint(self.arm_length, 17)
            self.coil_length = np.random.randint(6, self.window_length -5)
            self.window_width = self.arm_start - self.coil_start - self.coil_width - 1
            self.coil_turns = 500
            self.coil_current = 1

        else:               
            self.coil_start = 2
            self.arm_start = 27
            self.coil_width = 3
            self.coil2_start = 8 + 8 - (self.coil_start + self.coil_width)
            self.arm_width = 6
            self.arm_length = 15
            self.window_length = 15
            self.coil_length = 9
            self.window_width = 21
            self.coil_turns = np.random.choice(self.coil_turns_choice)
            self.coil_current = np.random.choice(self.coil_current_choice)

#        self.create_geo()
                
        #print('RESETing with a state[',self.posR,',',self.posC, '] = ', ob[self.posR, self.posC])        
        
        self.create_geo()        
        self.buffer = 3
        ob = np.ones(self.env_dim)*3

        ob[self.buffer : self.buffer + self.window_length, 
           self.coil_start + self.coil_width : 
               self.coil_start + self.coil_width + self.window_width] = 0

        ob[self.buffer: self.buffer + self.coil_length, 
           self.coil_start: self.coil_start + self.coil_width] = self.coil_turns * self.coil_current
           
        ob[self.buffer: self.buffer + self.coil_length , 
           self.coil2_start : self.coil2_start + self.coil_width] = self.coil_turns * self.coil_current

        ob[self.buffer: self.buffer + self.arm_length, 
           self.arm_start : self.arm_start + self.arm_width] = 4                       
           
#        ob = np.ones(self.env_dim)*3
#        ob[1:self.window_length + 1, self.coil_start + self.coil_width : 
#            self.coil_start + self.coil_width + self.window_width] = 0
#        ob[1:self.coil_length + 1, self.coil_start: self.coil_start + self.coil_width] = 2
#        
#        ob[1:self.coil_length + 1, self.coil2_start : self.coil2_start + self.coil_width] = 2
#        ob[1:self.arm_length + 1, self.arm_start : self.arm_start + self.arm_width] = 4
        
        #print('RESETing with a state[',self.posR,',',self.posC, '] = ', ob[self.posR, self.posC])
        
        self.posC = self.coil_start + self.coil_width + self.worm_step_size -2
        self.posR = self.worm_step_size + 1        
                  
        self.state_hist.append((np.copy(ob[self.buffer:-1, :])))        

        ob = self.move(ob)    
#        ob = self.clean_up(ob)
        self.past_states.append(np.copy(ob))
        self.past_actions.append(4)
        F = self.reward(ob)
        self.past_rewards.append(np.copy(F))
        warnings.filterwarnings("ignore")
        ob_image = np.copy(ob)
        ob_image[ob==self.coil_turns * self.coil_current] = 2
        imageio.imwrite(FEMM_PATH + 'first.png', img_as_uint(ob_image/5))
        self.frame_idx += 1
      #  print('reset done')
#        state_index = np.argmax(ob[1:16, 5:26])        
        
        ob2 = ob[self.buffer:, :]
        ob2[:, self.arm_start-1] = self.B_state[:, self.arm_start-1]
        
#        coil_indices = np.where(ob2 == 2)
#        self.AN_state = np.zeros_like(ob2)
#        self.AN_state[coil_indices] = self.coil_turns * self.coil_current
        
        #ob3 = np.dstack((ob2, np.copy(self.B_state)))        
        self.past_BnStates.append(np.copy(ob2))                
        ob3 = np.expand_dims(ob2, 0)

        ob4 = np.copy(ob3)
                      
        return(ob4)
        
    def render(self, train= False):

        if train == True:    
            eps_dir = FEMM_PATH + TRAIN_PATH + '\\Eps_' + str(self.frame_idx) + '\\'
        else:
            eps_dir = FEMM_PATH + MODEL_PATH + 'Eps_' + str(self.frame_idx) + '\\'   
        
        if not (os.path.exists(FEMM_PATH + MODEL_PATH)):
            os.mkdir(FEMM_PATH + MODEL_PATH)

        if not (os.path.exists(eps_dir)):
            os.mkdir(eps_dir)

        toP_image = np.copy(self.past_states[-1])
        toP_image[toP_image==self.coil_turns * self.coil_current] = 2                    

        toP_image = toP_image[self.buffer:-1, :]
#        toP[toP == 1] = 5
        toP_image = toP_image/np.max(toP_image)
        imageio.imwrite(eps_dir + str(self.step_count) + '_Issue-' + str(self.issue) + '_Action-' + str(self.past_actions[-1]) + '_reward-' + str(self.R) + '.png', img_as_uint(toP_image))
        
def save_file(src, fileName):
    dst = FEMM_PATH + MODEL_PATH + fileName + '.py'    
    copyfile(src, dst)

def get_file(path_file):
    path_components = path_file.split('\\')
    for component in path_components:
        file_sre = re.search('(.*).py', component)
#        file_name = file_sre.group(1)
    return file_sre     

def get_file_back_slash(path_file):
    path_components = path_file.split('/')
    for component in path_components:
        file_sre = re.search('(.*).py', component)
#        file_name = file_sre.group(1)
    return file_sre     

def create_txt_file(txt):
    with open(FEMM_PATH + MODEL_PATH + "history.txt", "w") as myfile:
        myfile.write(txt)
        
def append_txt_file(txt):
    with open(FEMM_PATH + MODEL_PATH + "history.txt", "a") as myfile:
            myfile.write(txt)
      
def save_obj(obj, name):    
    with open(FEMM_PATH + MODEL_PATH + name + '.pkl', 'wb') as f:        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)            

def load_obj(name ):    
    with open('obj/' + name + '.pkl', 'rb') as f:        
        return pickle.load(f)

In [0]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from env_Worm_Dy_NA_stacked import WormFemmEnv3
import constant_1N3 as con

env = WormFemmEnv3(max_steps=con.max_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device.type)

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(4, 16, kernel_size=3, padding=[1,1])
        self.bn1 = nn.BatchNorm2d(16)
        #self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=[1,1])
        #self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=[1,1], stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
#        def conv2d_size_out(size, kernel_size = 3, stride = 1):
#            return (size - (kernel_size - 1) - 1) // stride  + 1
#        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
#        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = int(h/2 * w/2 * 32)
#        print(h, w, linear_input_size, outputs)
        self.head1 = nn.Linear(linear_input_size, int(linear_input_size/2))                
        self.head2 = nn.Linear(int(linear_input_size/2), int(linear_input_size/4))
        self.tail = nn.Linear(int(linear_input_size/4), outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
#        print(x.shape)

        x = F.relu(self.bn1(self.conv1(x)))
        #x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
#        print(x.shape)
        x = F.relu(self.head1(x.view(x.size(0), -1)))
        x = F.relu(self.head2(x))
        return self.tail(x)
    
BATCH_SIZE = 512
GAMMA = 0.95
EPS_START = 0.95
EPS_END = 0.05
EPS_DECAY = 25
TARGET_UPDATE = 20

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
#init_screen = get_screen()
screen_height, screen_width = env.env_dim
print(screen_height, screen_width)
# Get number of actions from gym action space
n_actions = con.action_dim

policy_net = DQN(screen_height-3, screen_width, n_actions).to(device)
target_net = DQN(screen_height-3, screen_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(5000)
good_memory = ReplayMemory(5000)

def select_action(state):
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * env.frame_idx / EPS_DECAY)
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state.float()).max(1)[1].view(1, 1)
    else:
        env.exp_count += 1
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

#state = env.reset()
#state = torch.tensor(state, device=device)
#state = torch.unsqueeze(state, 0)        
#action = select_action(state)        
#next_state, reward, done, _ = env.step(action.item())                   

def optimize_model():
    if len(memory) < BATCH_SIZE and len(good_memory) < (3*env.max_steps):
        loss = 0.0
        return loss
    
    transitions = memory.sample(BATCH_SIZE)
    good_transitions = good_memory.sample(int(2*env.max_steps))
    new_transitions = transitions + good_transitions
    mix_transitions = random.sample(new_transitions, BATCH_SIZE)
    
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*mix_transitions))
    
    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.uint8)
    
    # Issue 'non_final_next_states' is torch.float32
    non_final_next_states = torch.cat([torch.unsqueeze(s, 0) for s in batch.next_state
                                                if s is not None])
             
#    state_batch = torch.cat(batch.state)
    state_batch = torch.cat([torch.unsqueeze(s, 0) for s in batch.state])
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)                              

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch.float()).gather(1, action_batch.view(-1,1).long())

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states.float()).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    return loss.item()

num_episodes = 500
episode_net_force = []
play_episode_net_force = []
i_episode = env.frame_idx
good_mem_avg = 0.0

for i_episode in range(num_episodes):
    # Initialize the environment and state
    reset_frame = env.reset()
    reset_frame = torch.tensor(reset_frame, device=device, dtype=torch.float64)
    state = torch.cat([reset_frame, reset_frame, reset_frame, reset_frame], dim=0)                      
              
    i_episode = env.frame_idx
    eps_loss = 0
    eps_memory = []
        
#    last_screen = get_screen()
#    current_screen = get_screen()
#    state = current_screen - last_screen
    for t in count():
        # Select and perform an action
#        print(state.shape)
        action = select_action(torch.unsqueeze(state, 0))
        next_frame, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        
        # Observe new state
#        last_screen = current_screen
#        current_screen = get_screen()
        if not done:
            next_frame = torch.tensor(next_frame, device=device, dtype=torch.float64)
            next_state = torch.cat([next_frame, state[0:-1, :,:]], dim=0)      
        else:
            next_state = None

        # Store the transition in memory
#        print(state.shape)
        
#        print(next_state.shape)
        memory.push(state, action, next_state, reward.float())
#        print(next_state.shape)
        eps_memory.append(Transition(state, action, next_state, reward.float()))

        # Move to the next state
        state = next_state
        #env.render(train=True)

        # Perform one step of the optimization (on the target network)
        loss = optimize_model()
        eps_loss += loss
        if done:
            print('Episode {:d}, Exp {:d}, Amp-N {:3.2f}, loss= {:2.3f} Net Force= {:2.5f}'
                  .format(env.frame_idx, env.exp_count, env.coil_turns*env.coil_current, eps_loss/env.step_count, env.net_force))
            episode_net_force.append(env.net_force)
            
            if (sum(episode_net_force)/len(episode_net_force)) <= env.net_force:
                for i in range(len(eps_memory)):
                    state, action, next_state, reward = eps_memory[i]
                    good_memory.push(state, action, next_state, reward)
                    
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
        
        reset_frame = env.reset()
        reset_frame = torch.tensor(reset_frame, device=device, dtype=torch.float64)           
        state = torch.cat([reset_frame, reset_frame, reset_frame, reset_frame], dim=0)                          
    
        for t in count():
        # Select and perform an action
#        print(state.shape)
            action = target_net(torch.unsqueeze(state, 0).float()).max(1)[1].view(1, 1)
            next_frame, reward, done, _ = env.step(action.item())
            reward = torch.tensor([reward], device=device)
        
        # Observe new state
#        last_screen = current_screen
#        current_screen = get_screen()
            if not done:                
                next_frame = torch.tensor(next_frame, device=device, dtype=torch.float64)
                next_state = torch.cat([next_frame, state[0:-1, :,:]], dim=0)                               
            else:
                next_state = None

        # Move to the next state
            state = next_state
            env.render(train=False)
            if done:
                print('\n \t Episode {:d}, Exp {:d}, Amp-N {:3.2f}, Net Force= {:2.5f}'
                      .format(env.frame_idx, env.exp_count, env.coil_turns*env.coil_current, env.net_force))
                play_episode_net_force.append(env.net_force)
                break
            
print('Complete')

Episode 501, Net Force= 24.91337
Episode 502, Net Force= 0.00332
Episode 503, Net Force= 0.05711
Episode 504, Net Force= 0.12188
Episode 505, Net Force= 21.92158
Episode 506, Net Force= 0.05147
Episode 507, Net Force= 0.41154
Episode 508, Net Force= 0.00972
Episode 509, Net Force= 0.01089
Episode 510, Net Force= 26.09894
Episode 511, Net Force= 0.01149
Episode 512, Net Force= 0.01738
Episode 513, Net Force= 0.00963
Episode 514, Net Force= 14.52533
Episode 515, Net Force= 0.03156
Episode 516, Net Force= 4.09244
Episode 517, Net Force= 0.16976
Episode 518, Net Force= 0.01262
Episode 519, Net Force= 1.90709
Episode 520, Net Force= 0.01386
Episode 521, Net Force= 0.08364
Episode 522, Net Force= 0.22431
Episode 523, Net Force= 4.89251
Episode 524, Net Force= 10.24755
Episode 525, Net Force= 19.67229
Episode 526, Net Force= 0.18723
Episode 527, Net Force= 0.02793
Episode 528, Net Force= 0.00302
Episode 529, Net Force= 0.03488
Episode 530, Net Force= 7.22705
Episode 531, Net Force= 0.00425
Ep